In [1]:
# Oxford pets 데이터셋으로 U-net 학습하기
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras import layers
import os
import random
import cv2 as cv

input_dir = './datasets/oxford_pets/images/images/'
target_dir = './datasets/oxford_pets/annotations/annotations/trimaps/'
img_siz = (160, 160)        # 모델에 입력되는 영상 크기
n_class = 3                 # 분할 레이블 (1:물체, 2:배경, 3:경계)
batch_siz = 8               # 미니 배치 크기

img_paths = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.jpg')])
label_paths = sorted([os.path.join(target_dir, f) for f in os.listdir(target_dir) if f.endswith('.png') and not f.startswith('.')])    ## endswith(): 문자열이 특정 문자열로 시작하는지 여부를 반환

class OxfordPets(keras.utils.Sequence):
    def __init__(self, batch_size, img_size, img_paths, label_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_paths = img_paths
        self.label_paths = label_paths
    
    def __len__(self):
        return len(self.img_paths) // self.batch_size
    
    def __getitem__(self, idx):
        i = idx * self.batch_size
        batch_img_paths = self.img_paths[i : i+self.batch_size]
        batch_label_paths = self.label_paths[i : i+self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype='float32')        # 입력 영상
        for j, path in enumerate(batch_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype='uint8')          # 분할 레이블
        for j, path in enumerate(batch_label_paths):
            img = load_img(path, target_size=self.img_size, color_mode='grayscale')
            y[j] = np.expand_dims(img, 2)
            y[j] -= 1           # 부류 번호를 1,2,3에서 0,1,2로 변경
        return x, y
    
def make_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))

    # U-net의 다운 샘플링(축소 경로)
    x = layers.Conv2D(32, 3, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    previous_block_activation = x               # 지름길 연결을 위해

    for filters in [64, 128, 256]:
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
        residual = layers.Conv2D(filters, 1, strides=2, padding='same')(previous_block_activation)
        x = layers.add([x, residual])           # 지름길 연결
        previous_block_activation = x           # 지름길 연결을 위해

    # U-net의 업 샘플링(확대 경로)
    for filters in [256, 128, 64, 32]:
        x = layers.Activation('relu')(x)
        x = layers.Conv2DTranspose(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2DTranspose(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.UpSampling2D(2)(x)
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding='same')(residual)
        x = layers.add([x, residual])           # 지름길 연결
        previous_block_activation = x           # 지름길 연결을 위해

    outputs = layers.Conv2D(num_classes, 3, activation='softmax', padding='same')(x)
    model = keras.Model(inputs, outputs)        # 모델 생성
    return model
    
model = make_model(img_siz, n_class)            # 모델 생성

random.Random(1).shuffle(img_paths)
random.Random(1).shuffle(label_paths)
test_samples = int(len(img_paths) * 0.1)        # 10%를 테스트 집합으로 사용
train_img_paths = img_paths[:-test_samples]
train_label_paths = label_paths[:-test_samples]
test_img_paths = img_paths[-test_samples:]
test_label_paths = label_paths[-test_samples:]

train_gen = OxfordPets(batch_siz, img_siz, train_img_paths, train_label_paths)      # 훈련 집합

test_gen = OxfordPets(batch_siz, img_siz, test_img_paths, test_label_paths)         # 검증 집합

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cb = [keras.callbacks.ModelCheckpoint('oxford_seg.h5', save_best_only=True)]        # 학습 결과 자동 저장

model.fit(train_gen, epochs=2, validation_data=test_gen, callbacks=cb)

preds = model.predict(test_gen)                 # 예측

cv.imshow('Sample image', cv.imread(test_img_paths[0]))                 # 0번 영상 디스플레이
cv.imshow('Segmentation label', cv.imread(test_label_paths[0])*64)
cv.imshow('Segmentation prediction', preds[0])                          # 0번 영상 예측 결과 디스플레이

cv.waitKey()
cv.destroyAllWindows()

Epoch 1/2
 90/831 [==>...........................] - ETA: 46s - loss: 2.2299 - accuracy: 0.2975

KeyboardInterrupt: 

In [1]:
# pixellib 라이브러리로 정지 영상을 의미 분할하기
from pixellib.semantic import semantic_segmentation
import cv2 as cv

seg = semantic_segmentation()
seg.load_ade20k_model('deeplabv3_xception65_ade20k.h5')

img_fname = 'busy_street.jpg'
seg.segmentAsAde20k(img_fname, output_image_name='image_new.jpg')
info1, img_segmented1 = seg.segmentAsAde20k(img_fname)
info2, img_segmented2 = seg.segmentAsAde20k(img_fname, overlay=True)

cv.imshow('Image original', cv.imread(img_fname))
cv.imshow('Image segmention', img_segmented1)
cv.imshow('Image segmention overlayed', img_segmented2)

cv.waitKey()
cv.destroyAllWindows()

Processed Image saved successfuly in your current working directory.


In [2]:
print(info1)

{'class_ids': array([  1,   2,   5,   7,  10,  12,  13,  14,  18,  21,  30,  41,  42,
        44,  55,  60,  81,  94, 101, 116, 128, 137, 139], dtype=int64), 'masks': array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])}


In [3]:
# pixellib 라이브러리로 비디오를 의미 분할하기
from pixellib.semantic import semantic_segmentation
import cv2 as cv

cap = cv.VideoCapture(0)

seg_video = semantic_segmentation()
seg_video.load_ade20k_model('deeplabv3_xception65_ade20k.h5')

seg_video.process_camera_ade20k(cap, overlay=True, frames_per_second=2, output_video_name='output_video.mp4',
                                show_frames=True, frame_name='Pixellib')

cap.release()
cv.destroyAllWindows()

In [1]:
# pixellib 라이브러리로 정지 영상을 사례 분할하기
from pixellib.instance import instance_segmentation
import cv2 as cv

seg = instance_segmentation()
seg.load_model('mask_rcnn_coco.h5')

img_fname = 'busy_street.jpg'
info, img_segmented = seg.segmentImage(img_fname, show_bboxes=True)

cv.imshow('Image segmention overlayed', img_segmented)

cv.waitKey()
cv.destroyAllWindows()

Instructions for updating:
Use fn_output_signature instead


C:\Users\prohe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [1]:
# pixellib 라이브러리로 비디오를 사례 분할하기
from pixellib.instance import instance_segmentation
import cv2 as cv

cap = cv.VideoCapture(0)

seg_video = instance_segmentation()
seg_video.load_model('mask_rcnn_coco.h5')

target_class = seg_video.select_target_classes(person=True, chair=True)
seg_video.process_camera(cap, segment_target_classes=target_class, frames_per_second=2, 
                         show_frames=True, frame_name='Pixellib', show_bboxes=True)

cap.release()
cv.destroyAllWindows()

Instructions for updating:
Use fn_output_signature instead


C:\Users\prohe\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


UnboundLocalError: local variable 'width' referenced before assignment

In [1]:
# pixellib 라이브러리를 활용해 내 맘대로 배경 바꾸기
import cv2 as cv
import numpy as np
from PyQt5.QtWidgets import *
import sys
from pixellib.tune_bg import alter_bg

class VideoSpecialEffect(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('배경을 내 맘대로')
        self.setGeometry(200, 200, 450, 100)

        videoButton = QPushButton('배경을 내 맘대로 켜기', self)
        self.pickCombo = QComboBox(self)
        self.pickCombo.addItems(['Origin', 'Blur(low)', 'Blur(moderate)', 'Blur(extreme)', 'Red', 'Green', 'Blue'])
        quitButton = QPushButton('Exit', self)

        videoButton.setGeometry(10, 10, 200, 30)
        self.pickCombo.setGeometry(210, 10, 110, 30)
        quitButton.setGeometry(340, 10, 100, 30)

        videoButton.clicked.connect(self.videoSpecialEffectFunction)
        quitButton.clicked.connect(self.quitFunction)

    def videoSpecialEffectFunction(self):
        self.cap = cv.VideoCapture(0, cv.CAP_DSHOW)
        if not self.cap.isOpened():
            sys.exit('카메라 연결 실패')

        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            pick_effect = self.pickCombo.currentIndex()
            if pick_effect == 0:
                special_img = frame
            elif pick_effect == 1:
                special_img = change_bg.blur_frame(frame, low=True, detect='person')
            elif pick_effect == 2:
                special_img = change_bg.blur_frame(frame, moderate=True, detect='person')
            elif pick_effect == 3:
                special_img = change_bg.blur_frame(frame, extreme=True, detect='person')
            elif pick_effect == 4:
                special_img = change_bg.color_frame(frame, colors=(255,0,0), detect='person')
            elif pick_effect == 5:
                special_img = change_bg.color_frame(frame, colors=(0,255,0), detect='person')
            elif pick_effect == 6:
                special_img = change_bg.color_frame(frame, colors=(0,0,255), detect='person')

            cv.imshow('Special effect', special_img)
            cv.waitKey(1)

    def quitFunction(self):
        self.cap.release()
        cv.destroyAllWindows()
        self.close()

change_bg = alter_bg(model_type='pb')
change_bg.load_pascalvoc_model('xception_pascalvoc.pb')

app = QApplication(sys.argv)
win = VideoSpecialEffect()
win.show()
app.exec_()

0